In [1]:
 #!kaggle datasets download -d sohier/large-purchases-by-the-state-of-ca

In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_13234/3509738684.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Prep Dataset ( No need to run, only for data prep )

In [2]:
import pandas as pd
import bamboolib as bam
import pickle

## Drop and Cast cols

In [3]:
df = pd.read_csv(r'../../data/PURCHASE ORDER DATA EXTRACT 2012-2015_0.csv', sep=',', decimal='.')
df = df.drop(columns=['Purchase Date', 'LPA Number', 'Purchase Order Number', 'Requisition Number', 'Sub-Acquisition Type', 'Acquisition Method', 'Sub-Acquisition Method', 'Supplier Code', 'Supplier Qualifications', 'CalCard', 'Location'])

In [4]:
# drop rows with any NAs
mask=df.isna().sum(axis=1)==0
df=df.loc[mask]
df.shape

(273036, 20)

In [5]:
cols_cast=['Normalized UNSPSC','Class','Family','Segment']
for col in cols_cast:
    df[col] = pd.to_numeric(df[col], downcast='integer', errors='coerce')

<img title="a title" alt="Alt text" src="UNSPSC_tree.png" width="500" height="600" style="float: left;">

## Create Labels Dictionary

In [6]:
labels_dic={}
cols=['Segment Title','Family Title','Class Title','Normalized UNSPSC']
for col in cols:
    labels_dic[col]= df.groupby([col]).agg(cnt=('Creation Date', 'size')).reset_index()
labels_dic['Collated']=df.groupby(['Segment Title', 'Family Title', 'Class Title', 'Normalized UNSPSC']).agg(cnt=('Creation Date', 'size')).reset_index()

In [7]:
for key,val in labels_dic.items():
    print(f'{key}\t:{val.shape}')

Segment Title	:(56, 2)
Family Title	:(408, 2)
Class Title	:(2293, 2)
Normalized UNSPSC	:(12483, 2)
Collated	:(12483, 5)


In [8]:
tt=labels_dic['Collated']
tt = tt.groupby(['Segment Title', 'Family Title']).agg(sum_cnt=('cnt', 'sum')).reset_index()
tt

                                         Segment Title  \
0       Apparel and Luggage and Personal Care Products   
1       Apparel and Luggage and Personal Care Products   
2       Apparel and Luggage and Personal Care Products   
3       Apparel and Luggage and Personal Care Products   
4       Apparel and Luggage and Personal Care Products   
..                                                 ...   
403  Travel and Food and Lodging and Entertainment ...   
404  Travel and Food and Lodging and Entertainment ...   
405  Travel and Food and Lodging and Entertainment ...   
406  Travel and Food and Lodging and Entertainment ...   
407  Travel and Food and Lodging and Entertainment ...   

                                  Family Title  sum_cnt  
0                                     Clothing     2347  
1                                     Footwear      488  
2     Luggage and handbags and packs and cases      244  
3                       Personal care products      852  
4            

## Further cleansing

In [9]:
cols_drop=['Creation Date','Supplier Zip Code','Classification Codes','Total Price','Segment','Family','Class']
df.drop(labels=cols_drop,inplace=True,axis=1)
df.shape

(273036, 13)

In [10]:
df['Acquisition Type'].value_counts()

NON-IT Goods             172881
NON-IT Services           52514
IT Goods                  38272
IT Services                9243
IT Telecommunications       126
Name: Acquisition Type, dtype: int64

In [12]:
labels_dic['Acquisition Type']=df['Acquisition Type'].value_counts().index.tolist()
labels_dic.keys()

dict_keys(['Segment Title', 'Family Title', 'Class Title', 'Normalized UNSPSC', 'Collated', 'Acquisition Type'])

## Export and Save output

In [10]:
# df.to_csv('..\..\data\large-purchases-by-the-state-of-ca\cleansed_kaggle_po.csv',index=False)

In [14]:
import pickle
pickle_filename = '../../data/labels_dic.pkl'

# Open the file in binary write mode and dump the dictionary into it
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(labels_dic, pickle_file)
    
pickle_filename = '../../data/cleansed_kaggle_po.pkl'

# Open the file in binary write mode and dump the dictionary into it
with open(pickle_filename, 'wb') as pickle_file:
    pickle.dump(df, pickle_file)